In [2]:
using JuMP
using GLPK 
using Cbc

## Problema de Bin Packing

El problema de bin packing es un problema clásico de optimización combinatoria. Dado un conjunto de ítems con pesos y una capacidad máxima por contenedor, el objetivo es minimizar el número de contenedores utilizados para almacenar todos los ítems sin exceder la capacidad de ningún contenedor.

### Definiciones

- \( n \): Número total de ítems.
- \( \text{items}[j] \): Peso del ítem \( j \), para \( j = 1, \ldots, n \).
- \( \text{bin\_capacity} \): Capacidad máxima de cada contenedor.

### Variables de Decisión

- \( x_{ij} \): Variable binaria que indica si el ítem \( j \) está en el contenedor \( i \).
- \( y_i \): Variable binaria que indica si el contenedor \( i \) está en uso.

### Restricciones

1. **Cada ítem debe estar en exactamente un contenedor**:
   \[
   \sum_{i=1}^{\text{max\_bins}} x_{ij} = 1 \quad \forall j = 1, \ldots, n
   \]

2. **La suma de los pesos de los ítems en cada contenedor no debe exceder la capacidad del contenedor**:
   \[
   \sum_{j=1}^{n} x_{ij} \cdot \text{items}[j] \leq \text{bin\_capacity} \cdot y_i \quad \forall i = 1, \ldots, \text{max\_bins}
   \]

### Función Objetivo

Minimizar el número de contenedores utilizados:
\[
\text{Minimizar} \quad \sum_{i=1}^{\text{max\_bins}} y_i
\]


In [3]:
function bin_packing_glpk(items, bin_capacity)
    model = Model(GLPK.Optimizer)
    n = length(items)
    max_bins = n  # En el peor de los casos, cada ítem puede ir en un contenedor separado
    
    # Variables
    @variable(model, x[1:max_bins, 1:n], Bin)
    @variable(model, y[1:max_bins], Bin)

    # Restricciones
    @constraint(model, [j=1:n], sum(x[i,j] for i in 1:max_bins) == 1)
    @constraint(model, [i=1:max_bins], sum(x[i,j] * items[j] for j in 1:n) <= bin_capacity * y[i])

    # Función objetivo
    @objective(model, Min, sum(y))

    optimize!(model)

    println("Status: ", termination_status(model))
    println("Objective value: ", objective_value(model))
    println("Número de contenedores utilizados: ", sum(value.(y)))
    
    return value.(y)
end

bin_packing_glpk (generic function with 1 method)

In [4]:
items = [5, 10, 15, 20, 25, 30, 5, 2, 3, 4, 5, 20, 30]  
bin_capacity = 60 

# Ejecutar el algoritmo
num_bins_used = bin_packing_glpk(items, bin_capacity)
println("Número de contenedores utilizados: $(sum(num_bins_used))")

Status: OPTIMAL
Objective value: 3.0
Número de contenedores utilizados: 3.0
Número de contenedores utilizados: 3.0


In [5]:
items = [3,2,3,2,2,2]
bin_capacity = 10 

num_bins_used = bin_packing_glpk(items, bin_capacity)
println("Número de contenedores utilizados: $(sum(num_bins_used))")

Status: OPTIMAL
Objective value: 2.0
Número de contenedores utilizados: 2.0
Número de contenedores utilizados: 2.0


In [6]:
items = [2,5,4,7,1,3,8]
bin_capacity = 10 

num_bins_used = bin_packing_glpk(items, bin_capacity)
println("Número de contenedores utilizados: $(sum(num_bins_used))")

Status: OPTIMAL
Objective value: 3.0
Número de contenedores utilizados: 3.0
Número de contenedores utilizados: 3.0


In [7]:
function bin_packing_cbc(bin_capacity, items)
    n = length(items)
    max_bins = n  # En el peor de los casos, cada ítem puede ir en un contenedor separado
    
    model = Model(Cbc.Optimizer)
    
    # Variables
    @variable(model, x[1:max_bins, 1:n], Bin)
    @variable(model, y[1:max_bins], Bin)

    # Restricciones
    @constraint(model, [j=1:n], sum(x[i,j] for i in 1:max_bins) == 1)
    @constraint(model, [i=1:max_bins], sum(x[i,j] * items[j] for j in 1:n) <= bin_capacity * y[i])

    # Función objetivo
    @objective(model, Min, sum(y))

    set_optimizer_attribute(model, "logLevel", 0)  
    
    elapsed_time = @elapsed begin
        optimize!(model)
    end
    
    if termination_status(model) == MOI.INFEASIBLE
        println("El problema no tiene solución factible.")
        return -1  # Otra forma de indicar que no se encontró solución
    end
    
    num_bins_used = sum(value.(y))
    
    return num_bins_used, elapsed_time

end

bin_packing_cbc (generic function with 1 method)

In [8]:
# Función para leer y describir todas las instancias de un archivo
function describe_problems(problems)
    for (problem_id, bin_capacity, num_items, best_known_solution, items) in problems
        println("Resolviendo problema: $problem_id, con $num_items de items, se espera $best_known_solution, $items items")
    end
    return problems
end

describe_problems (generic function with 1 method)

In [9]:
function solve_bin_packing_instances(problems)
    for (problem_id, bin_capacity, num_items, best_known_solution, items) in problems
        println("Resolviendo problema: $problem_id, con $bin_capacity de capacidad y con $num_items de items, se espera solución óptima: $best_known_solution, $items")
        num_bins_used, time_taken = bin_packing_cbc(bin_capacity, items)
        println("Número de contenedores utilizados: $num_bins_used")
        println("Tiempo tomado: $time_taken segundos")
    end
end

solve_bin_packing_instances (generic function with 1 method)

In [10]:
# Función que permite leer las instancias de bin packing desde un archivo
function read_bin_packing_instance(filename)
    problems = []
    open(filename, "r") do file
        lines = readlines(file)
        P = parse(Int, lines[1])  # Número de instancias de problemas
        index = 2
        for _ in 1:P
            problem_id = strip(lines[index])
            index += 1
            bin_capacity, num_items, best_known_solution = split(lines[index])
            bin_capacity = parse(Float64, bin_capacity)
            num_items = parse(Int, num_items)
            best_known_solution = parse(Int, best_known_solution)
            index += 1
            items = []
            for _ in 1:num_items
                push!(items, parse(Float64, strip(lines[index])))
                index += 1
            end
            push!(problems, (problem_id, bin_capacity, num_items, best_known_solution, items))
        end
    end
    return problems
end

read_bin_packing_instance (generic function with 1 method)

In [9]:
problems = read_bin_packing_instance("./instances/uniform.txt")
solve_bin_packing_instances(problems)


Resolviendo problema: uniform1, con 150.0 de capacidad y con 10 de items, se espera solución óptima: 2, Any[25.0, 30.0, 35.0, 20.0, 40.0, 25.0, 30.0, 35.0, 25.0, 25.0]
Número de contenedores utilizados: 2.0
Tiempo tomado: 2.345512117 segundos
Resolviendo problema: uniform2, con 100.0 de capacidad y con 8 de items, se espera solución óptima: 2, Any[25.0, 25.0, 20.0, 30.0, 25.0, 20.0, 30.0, 25.0]
Número de contenedores utilizados: 2.0
Tiempo tomado: 0.013359444 segundos
Resolviendo problema: uniform3, con 120.0 de capacidad y con 6 de items, se espera solución óptima: 2, Any[30.0, 20.0, 30.0, 20.0, 10.0, 10.0]
Número de contenedores utilizados: 1.0
Tiempo tomado: 0.008336982 segundos
Resolviendo problema: uniform4, con 100.0 de capacidad y con 10 de items, se espera solución óptima: 4, Any[25.0, 20.0, 25.0, 20.0, 25.0, 20.0, 20.0, 25.0, 20.0, 25.0]
Número de contenedores utilizados: 3.0
Tiempo tomado: 0.018188245 segundos
Resolviendo problema: uniform5, con 80.0 de capacidad y con 12 de 

In [12]:
problems = read_bin_packing_instance("./instances/uniform2.txt")
solve_bin_packing_instances(problems)

Resolviendo problema: uniform1_2, con 300.0 de capacidad y con 20 de items, se espera solución óptima: 2, Any[20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
Número de contenedores utilizados: 1.0
Tiempo tomado: 2.791874861 segundos
Resolviendo problema: uniform2_2, con 180.0 de capacidad y con 20 de items, se espera solución óptima: 5, Any[20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
Número de contenedores utilizados: 2.0
Tiempo tomado: 0.105924495 segundos
Resolviendo problema: uniform3_2, con 150.0 de capacidad y con 20 de items, se espera solución óptima: 5, Any[20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
Número de contenedores utilizados: 2.0
Tiempo tomado: 0.130918243 segundos
Resolviendo problema: uniform4_2, con 120.0 de capacidad y con 20 de items, se espera

In [12]:
problems = read_bin_packing_instance("./instances/triplets.txt")
solve_bin_packing_instances(problems)

Resolviendo problema: triplet1, con 100.0 de capacidad y con 9 de items, se espera solución óptima: 3, Any[25.0, 25.0, 25.0, 20.0, 20.0, 20.0, 15.0, 15.0, 10.0]
Número de contenedores utilizados: 2.0
Tiempo tomado: 0.005857133 segundos
Resolviendo problema: triplet2, con 80.0 de capacidad y con 6 de items, se espera solución óptima: 2, Any[25.0, 25.0, 15.0, 20.0, 20.0, 10.0]
Número de contenedores utilizados: 2.0
Tiempo tomado: 0.005577273 segundos
Resolviendo problema: triplet3, con 100.0 de capacidad y con 9 de items, se espera solución óptima: 3, Any[30.0, 30.0, 30.0, 20.0, 20.0, 20.0, 15.0, 15.0, 10.0]
Número de contenedores utilizados: 2.0
Tiempo tomado: 0.005819606 segundos
Resolviendo problema: triplet4, con 80.0 de capacidad y con 6 de items, se espera solución óptima: 2, Any[25.0, 25.0, 15.0, 15.0, 20.0, 20.0]
Número de contenedores utilizados: 2.0
Tiempo tomado: 0.004684833 segundos
Resolviendo problema: triplet5, con 100.0 de capacidad y con 9 de items, se espera solución óp

In [15]:
problems = read_bin_packing_instance("./instances/instances2.txt")
solve_bin_packing_instances(problems)

Resolviendo problema: instance30, con 100.0 de capacidad y con 30 de items, se espera solución óptima: 6, Any[29.7, 36.9, 25.8, 38.5, 27.0, 23.6, 22.2, 26.1, 23.2, 22.4, 39.9, 30.3, 27.5, 36.9, 24.9, 20.3, 22.6, 39.0, 32.6, 21.1, 28.6, 23.8, 34.3, 37.9, 37.6, 37.3, 38.7, 25.5, 25.2, 27.8]
Número de contenedores utilizados: 9.0
Tiempo tomado: 0.568334997 segundos
Resolviendo problema: instance32, con 100.0 de capacidad y con 32 de items, se espera solución óptima: 6, Any[28.1, 37.0, 26.7, 23.6, 25.7, 37.2, 33.6, 35.8, 25.2, 38.6, 27.3, 23.8, 34.8, 32.7, 36.2, 30.0, 30.7, 23.2, 23.2, 36.8, 20.6, 38.8, 39.0, 29.5, 38.1, 21.4, 21.2, 34.9, 26.2, 34.3, 34.9, 37.3]
Número de contenedores utilizados: 10.0
Tiempo tomado: 0.682398347 segundos
Resolviendo problema: instance34, con 100.0 de capacidad y con 34 de items, se espera solución óptima: 6, Any[39.5, 34.7, 39.1, 38.6, 30.2, 32.6, 37.3, 39.7, 36.9, 30.9, 37.5, 24.0, 27.1, 35.9, 23.7, 22.8, 35.7, 38.5, 26.9, 34.6, 22.8, 23.4, 33.0, 24.9, 33.

In [17]:
problems = read_bin_packing_instance("./instances/instances3.txt")
solve_bin_packing_instances(problems)